#### Date of Scraping: April 28, 2019

### Importing libraries

In [1]:
import requests 
from bs4 import BeautifulSoup
import time
import re
import json

### Creating CSV file to be written¶

In [ ]:
with open("deathlist.csv", "w") as f:
    f.write("CaseNumber,Name,BirthDate,DeathDate,Age,Gender,DeathPlace,BodyStatus,Mode,Investigator,DepMedicalExaminer,CaseStatus,Race,CauseA,CauseB,CauseC,CauseD,CauseOther,toxstatus\n" )

### Defining function that starts with the first JSON file containing a list, scrapes all the details, writes it into the CSV, and moves on to the next CSV

In [ ]:
# defining function 
def pageDetails(url):   
    # takes the URL
    page = requests.get(url)
    # takes the JSON file from the URL
    data = page.json()
    # stores the data 
    records = data["cases"]
    # loops through each entry in the JSON
    for record in records:
            # stores the case number
            casenumber = record["CaseNum"]
            # checks if the person died after 2014
            deathdate = record["DeathDate"]
            if int(deathdate[6:]) > 2014:
                time.sleep(1)
                # stores the name
                name = str(record["NameFirst"]) + " " + str(record["NameLast"])
                name = name.replace(",", "-")
                # stores the birthdate
                birthdate = record["BirthDate"]
                # stores the age 
                age = record["Age"]
                # stores the gender
                gender = record["Gender"]
                # stores the place of death
                deathplace = record["DeathPlace"].replace(",", "-")
                # stores the status of the body
                bodystatus = record["BodyStatus"].replace(",", "-")
                # stores the mode of death
                mode = record["Mode"].replace(",", "-")
                # stores the name of the primary investigator
                investigator = record["Investigator"].replace(",", "-")
                # stores the name of the deputy medical examiner
                depmedexaminer = record["DME"].replace(",", "-")
                # stores the status of the case
                casestatus = record["CaseStatus"].replace(",", "-")
                # stores URL for the JSON file that stores further details of each case
                newurl = "http://api.lacounty.gov/mecsearch/CaseInformationServlet?caseDetails=1&CaseNum=" + str(casenumber)
                # takes the URL 
                detailpage = requests.get(newurl)
                # takes the JSON file from the URL
                detdata = detailpage.json()
                # stores the data 
                details = detdata["caseDetail"]
                # stores the race
                race = details[0]["Race"].replace(",", "-")
                # stores each cause
                causeA = details[0]["CauseA"].replace(",", "-")
                causeC = details[0]["CauseB"].replace(",", "-")
                causeB = details[0]["CauseC"].replace(",", "-")
                causeD = details[0]["CauseD"].replace(",", "-")
                causeOther = details[0]["CauseOther"].replace(",", "-")
                # stores the toxic status
                toxstatus = details[0]["ToxStatus"].replace(",", "-")
                # writes details into the CSV
                with open("deathlist.csv", "a") as f:
                            f.write(casenumber + "," + name + "," + birthdate + "," + deathdate + "," + age + "," + gender + "," + deathplace + "," + bodystatus + "," + mode + "," + investigator + "," + depmedexaminer + "," + casestatus + "," + race + "," + causeA + "," + causeB + "," + causeC + "," + causeD + "," + causeOther + "," + toxstatus + "\n") 
            else:
                break

### Creating a loop that starts with the initial URL of the first JSON file and moves through each one, calling the pre-defined function and writing into the CSV

In [ ]:
# runs through a loop that goes through each JSON file that contains lists of the dead
for offset in range(0, 1189):
    # starts with the initial loop, resetting each time reading that page is completed
    url = "http://api.lacounty.gov/mecsearch/CaseInformationServlet?pageNumber=" + str(offset) + "&pageSize=100&NameFirst=&NameLast=&BirthDate=&Age=&DeathDate=&CaseNum=&sortColumn=CaseNum&sortOrder=desc"
    # calls the function to write the details into the CSV
    pageDetails(url)
    time.sleep(1)

### Importing Libraries

In [2]:
import pandas as pd

### Opening the CSV file

In [3]:
# before importing the CSV, make sure the date is in MM/DD/YYYY format in Excel
deaths = pd.read_csv("deathlist.csv")
# displays a snapshot of the file 
deaths.head(5)

,CaseNumber,Name,BirthDate,DeathDate,Age,Gender,DeathPlace,BodyStatus,Mode,Investigator,DepMedExaminer,CaseStatus,Race,CauseA,CauseB,CauseC,CauseD,CauseOther,toxstatus
0,2019-03294,ELLEN CAVALLI,09/02/1943,04/29/2019,75.0,FEMALE,FREEWAY,HERE,NaN,LOPEZ,NaN,INV. COMPLETE,CAUCASIAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED
1,2019-03289,PAUL DURKIN,01/16/1994,04/28/2019,25.0,MALE,RESIDENCE,HERE,NaN,GUILMETTE,NaN,INV. ASSIGNED,CAUCASIAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED
2,2019-03284,TERRY MILLS,06/13/1952,04/28/2019,66.0,MALE,RESIDENCE,HERE,NaN,MUNOZ,NaN,INV. ASSIGNED,CAUCASIAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED
3,2019-03281,LORIBETH LOPEZ,07/11/1990,04/28/2019,28.0,FEMALE,STREET,HERE,NaN,EPPERSON,NaN,INV. COMPLETE,HISPANIC/LATIN AMERICAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED
4,2019-03279,RICARDO SEGURA,04/03/1997,04/27/2019,22.0,MALE,VEHICLE,EXAM SCHEDULED,NaN,REYNOLDS,NaN,Exam Pending,HISPANIC/LATIN AMERICAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED


In [4]:
# function to return new date format in YYYY-MM-DD
def DateFormat(x):
    # ensures that the data-type is a string
    x = str(x)
    newdate = x[6:] + "-" + x[:2] + "-" + x[3:][:2]
    return newdate   

In [5]:
# creates new column with reformatted dates
deaths["DateOfDeath"] = deaths["DeathDate"].apply(lambda x: DateFormat(x))
deaths["DateOfBirth"] = deaths["BirthDate"].apply(lambda x: DateFormat(x))   

In [6]:
deaths.head(5)

,CaseNumber,Name,BirthDate,DeathDate,Age,Gender,DeathPlace,BodyStatus,Mode,Investigator,...,CaseStatus,Race,CauseA,CauseB,CauseC,CauseD,CauseOther,toxstatus,DateOfDeath,DateOfBirth
0,2019-03294,ELLEN CAVALLI,09/02/1943,04/29/2019,75.0,FEMALE,FREEWAY,HERE,NaN,LOPEZ,...,INV. COMPLETE,CAUCASIAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED,2019-04-29,1943-09-02
1,2019-03289,PAUL DURKIN,01/16/1994,04/28/2019,25.0,MALE,RESIDENCE,HERE,NaN,GUILMETTE,...,INV. ASSIGNED,CAUCASIAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED,2019-04-28,1994-01-16
2,2019-03284,TERRY MILLS,06/13/1952,04/28/2019,66.0,MALE,RESIDENCE,HERE,NaN,MUNOZ,...,INV. ASSIGNED,CAUCASIAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED,2019-04-28,1952-06-13
3,2019-03281,LORIBETH LOPEZ,07/11/1990,04/28/2019,28.0,FEMALE,STREET,HERE,NaN,EPPERSON,...,INV. COMPLETE,HISPANIC/LATIN AMERICAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED,2019-04-28,1990-07-11
4,2019-03279,RICARDO SEGURA,04/03/1997,04/27/2019,22.0,MALE,VEHICLE,EXAM SCHEDULED,NaN,REYNOLDS,...,Exam Pending,HISPANIC/LATIN AMERICAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED,2019-04-27,1997-04-03


In [7]:
# saving as file
deaths.to_csv("newdeathlist.csv", index=False)

In [8]:
# imports file with the new date format
newdeaths = pd.read_csv("newdeathlist.csv")

In [9]:
newdeaths.head(5)

,CaseNumber,Name,BirthDate,DeathDate,Age,Gender,DeathPlace,BodyStatus,Mode,Investigator,...,CaseStatus,Race,CauseA,CauseB,CauseC,CauseD,CauseOther,toxstatus,DateOfDeath,DateOfBirth
0,2019-03294,ELLEN CAVALLI,09/02/1943,04/29/2019,75.0,FEMALE,FREEWAY,HERE,NaN,LOPEZ,...,INV. COMPLETE,CAUCASIAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED,2019-04-29,1943-09-02
1,2019-03289,PAUL DURKIN,01/16/1994,04/28/2019,25.0,MALE,RESIDENCE,HERE,NaN,GUILMETTE,...,INV. ASSIGNED,CAUCASIAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED,2019-04-28,1994-01-16
2,2019-03284,TERRY MILLS,06/13/1952,04/28/2019,66.0,MALE,RESIDENCE,HERE,NaN,MUNOZ,...,INV. ASSIGNED,CAUCASIAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED,2019-04-28,1952-06-13
3,2019-03281,LORIBETH LOPEZ,07/11/1990,04/28/2019,28.0,FEMALE,STREET,HERE,NaN,EPPERSON,...,INV. COMPLETE,HISPANIC/LATIN AMERICAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED,2019-04-28,1990-07-11
4,2019-03279,RICARDO SEGURA,04/03/1997,04/27/2019,22.0,MALE,VEHICLE,EXAM SCHEDULED,NaN,REYNOLDS,...,Exam Pending,HISPANIC/LATIN AMERICAN,NaN,NaN,NaN,NaN,NaN,NOT REQUESTED,2019-04-27,1997-04-03


### Filters for deaths in schools

In [10]:
schooldeaths = newdeaths[newdeaths["DeathPlace"] == "SCHOOL/ COLLEGE"]

In [11]:
schooldeaths.shape

(32, 21)

In [12]:
schooldeaths

,CaseNumber,Name,BirthDate,DeathDate,Age,Gender,DeathPlace,BodyStatus,Mode,Investigator,...,CaseStatus,Race,CauseA,CauseB,CauseC,CauseD,CauseOther,toxstatus,DateOfDeath,DateOfBirth
400,2019-02577,SANDRA MENDOZA MUNGIA,07/27/1967,04/01/2019,51.0,FEMALE,SCHOOL/ COLLEGE,RELEASED,NaN,MUNOZ,...,DEFERRED Pending Additional Investigation,CAUCASIAN,DEFERRED,NaN,NaN,NaN,NaN,REQUESTED,2019-04-01,1967-07-27
951,2019-01649,ERIC CRAMER,04/23/1996,02/26/2019,22.0,MALE,SCHOOL/ COLLEGE,RELEASED,SUICIDE,DEMING,...,CLOSED,CAUCASIAN,GUNSHOT WOUND TO HEAD,NaN,NaN,NaN,BIPOLAR DISORDER,REQUESTED,2019-02-26,1996-04-23
1092,2019-01404,JEREMY PETERSON,06/27/1996,02/19/2019,22.0,MALE,SCHOOL/ COLLEGE,READY,ACCIDENT,SALGADO,...,CLOSED,ASIAN,COMBINED EFFECTS OF FENTANYL INTOXICATION AND ...,NaN,NaN,NaN,HYPERTROPHIC HEART DISEASE,REQUESTED,2019-02-19,1996-06-27
1470,2019-00856,GERALD MCKEON,05/17/1947,01/31/2019,71.0,MALE,SCHOOL/ COLLEGE,READY,NATURAL,LOPEZ,...,CLOSED,CAUCASIAN,MYOCARDIAL INFARCTION,ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE,THROMBOSIS OF RIGHT AND LEFT CORONARY ARTERIES,NaN,NaN,REQUESTED,2019-01-31,1947-05-17
2382,2018-09353,DOUGLAS CAMPBELL,01/18/1972,12/09/2018,46.0,MALE,SCHOOL/ COLLEGE,RELEASED,SUICIDE,FRAZIER,...,CLOSED,CAUCASIAN,GUNSHOT WOUND OF HEAD,NaN,NaN,NaN,NaN,REQUESTED,2018-12-09,1972-01-18
2835,2018-08670,JESUS LOPEZ PULIDO,06/15/1955,11/14/2018,63.0,MALE,SCHOOL/ COLLEGE,READY,NATURAL,BRANSON,...,CLOSED,HISPANIC/LATIN AMERICAN,CORONARY ARTERY ATHEROSCLEROSIS,NaN,ATHEROSCLEROTIC CARDIOVASCULAR DISEASE,NaN,DIABETES MELLITUS AND DILATED CARDIOMYOPATHY D...,REQUESTED,2018-11-14,1955-06-15
3771,2018-07281,KEVIN KING,06/05/1994,09/20/2018,24.0,MALE,SCHOOL/ COLLEGE,RELEASED,SUICIDE,REYNOLDS,...,CLOSED,HISPANIC/LATIN AMERICAN,GUNSHOT WOUND OF HEAD,NaN,NaN,NaN,NaN,REQUESTED,2018-09-20,1994-06-05
4131,2018-06746,LAWRENCE VUONG,12/01/1999,08/31/2018,18.0,MALE,SCHOOL/ COLLEGE,RELEASED,SUICIDE,REYNOLDS,...,CLOSED,ASIAN,GUNSHOT WOUND OF THE HEAD,NaN,NaN,NaN,NaN,REQUESTED,2018-08-31,1999-12-01
5398,2018-04790,DANIEL CALLAHAN,07/13/1963,06/23/2018,54.0,MALE,SCHOOL/ COLLEGE,RELEASED,SUICIDE,DIAZ,...,CLOSED,CAUCASIAN,GUNSHOT WOUND OF THE CHEST,NaN,NaN,NaN,NaN,REQUESTED,2018-06-23,1963-07-13
5614,2018-04399,FRANCISCO PENADO JR.,11/03/1973,06/09/2018,44.0,MALE,SCHOOL/ COLLEGE,RELEASED,HOMICIDE,MCCRACKEN,...,CLOSED,HISPANIC/LATIN AMERICAN,GUNSHOT WOUND OF THE HEAD,NaN,NaN,NaN,NaN,REQUESTED,2018-06-09,1973-11-03
